In [6]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import census_api_key
c = Census(census_api_key, year=2014)

In [9]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/datamade/census for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B08301_001E", "B01003_001E", "B01002_001E", "B08301_010E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B08301_001E": "Transportation (total)",
                                      "B08301_010E": "Public Transportation",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Public Transportation Rate (Public Transportation / Transportation (total))
census_pd["Public Transportation Rate"] = 100 * \
    census_pd["Public Transportation"].astype(
        int) / census_pd["Transportation (total)"].astype(int)


# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Transportation (total)",
                       "Public Transportation","Public Transportation Rate"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Median Age,Transportation (total),Public Transportation,Public Transportation Rate
0,01740,5019.0,42.7,2623.0,72.0,2.744949
1,01741,4967.0,48.5,2211.0,14.0,0.633198
2,01742,18948.0,46.7,7623.0,571.0,7.490489
3,01745,309.0,50.4,162.0,15.0,9.259259
4,01746,14008.0,42.6,6904.0,260.0,3.765933


In [11]:
census_pd.to_csv("Resources/census_data.csv", encoding="utf-8", index=False)